#### Mục tiêu: Xây dựng mô hình có độ chính xác trên tập validation khoảng 94%

In [1]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

!wget --no-check-certificate \
    https://storage.googleapis.com/protonx-cloud-storage/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2020-07-10 13:39:52--  https://storage.googleapis.com/protonx-cloud-storage/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.184.128, 64.233.167.128, 74.125.71.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.184.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.05s   

2020-07-10 13:39:52 (107 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [45]:
vocab_size =  1000 # FIXME
embedding_dim = 16 # FIXME
max_length = 120 # FIXME
trunc_type = 'post' # FIXME
padding_type = 'post' # FIXME
oov_tok = '<OOV>'# FIXME
training_portion = .8

In [2]:
sentences = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
print(len(stopwords))
# Expected Output
# 153

153


In [7]:
with open("/tmp/bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
        sentences.append(sentence)

print(len(labels))
print(len(sentences))
print(sentences[0])
# Expected Output
# 2225
# 2225
# tv future hands viewers home theatre systems  plasma high-definition tvs  digital video recorders moving living room  way people watch tv will radically different five years  time.  according expert panel gathered annual consumer electronics show las vegas discuss new technologies will impact one favourite pastimes. us leading trend  programmes content will delivered viewers via home networks  cable  satellite  telecoms companies  broadband service providers front rooms portable devices.  one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes  like us s tivo uk s sky+ system  allow people record  store  play  pause forward wind tv programmes want.  essentially  technology allows much personalised tv. also built-in high-definition tv sets  big business japan us  slower take off europe lack high-definition programming. not can people forward wind adverts  can also forget abiding network channel schedules  putting together a-la-carte entertainment. us networks cable satellite companies worried means terms advertising revenues well  brand identity  viewer loyalty channels. although us leads technology moment  also concern raised europe  particularly growing uptake services like sky+.  happens today  will see nine months years  time uk   adam hume  bbc broadcast s futurologist told bbc news website. likes bbc  no issues lost advertising revenue yet. pressing issue moment commercial uk broadcasters  brand loyalty important everyone.  will talking content brands rather network brands   said tim hanlon  brand communications firm starcom mediavest.  reality broadband connections  anybody can producer content.  added:  challenge now hard promote programme much choice.   means  said stacey jolna  senior vice president tv guide tv group  way people find content want watch simplified tv viewers. means networks  us terms  channels take leaf google s book search engine future  instead scheduler help people find want watch. kind channel model might work younger ipod generation used taking control gadgets play them. might not suit everyone  panel recognised. older generations comfortable familiar schedules channel brands know getting. perhaps not want much choice put hands  mr hanlon suggested.  end  kids just diapers pushing buttons already - everything possible available   said mr hanlon.  ultimately  consumer will tell market want.   50 000 new gadgets technologies showcased ces  many enhancing tv-watching experience. high-definition tv sets everywhere many new models lcd (liquid crystal display) tvs launched dvr capability built  instead external boxes. one example launched show humax s 26-inch lcd tv 80-hour tivo dvr dvd recorder. one us s biggest satellite tv companies  directtv  even launched branded dvr show 100-hours recording capability  instant replay  search function. set can pause rewind tv 90 hours. microsoft chief bill gates announced pre-show keynote speech partnership tivo  called tivotogo  means people can play recorded programmes windows pcs mobile devices. reflect increasing trend freeing multimedia people can watch want  want.

2225
2225
tv future hands viewers home theatre systems  plasma high-definition tvs  digital video recorders moving living room  way people watch tv will radically different five years  time.  according expert panel gathered annual consumer electronics show las vegas discuss new technologies will impact one favourite pastimes. us leading trend  programmes content will delivered viewers via home networks  cable  satellite  telecoms companies  broadband service providers front rooms portable devices.  one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes  like us s tivo uk s sky+ system  allow people record  store  play  pause forward wind tv programmes want.  essentially  technology allows much personalised tv. also built-in high-definition tv sets  big business japan us  slower take off europe lack high-definition programming. not can people forward wind adverts  can also forget abiding network channel schedules  putting together a-la-carte entertai

In [ ]:
labels


In [16]:
train_size = int(len(sentences) * training_portion) # FIXME

train_sentences = sentences[:train_size] # FIXME
train_labels = labels[:train_size] # FIXME

validation_sentences = sentences[train_size:]# FIXME
validation_labels = labels[train_size:] # FIXME

print(train_size)
print(len(train_sentences))
print(len(train_labels))
print(len(validation_sentences))
print(len(validation_labels))

# Expected output (if training_portion=.8)
# 1780
# 1780
# 1780
# 445
# 445

1780
1780
1780
445
445


In [46]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok) # FIXME
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences =  tokenizer.texts_to_sequences(train_sentences) # FIXME
train_padded = pad_sequences(train_sequences, maxlen=max_length, 
                                       truncating=trunc_type, padding=padding_type)# FIXME

print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

# Expected Ouput
# 449
# 120
# 200
# 120
# 192
# 120

449
120
200
120
192
120


In [18]:
validation_sequences = tokenizer.texts_to_sequences(validation_sentences) # FIXME
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, 
                                       truncating=trunc_type, padding=padding_type)# FIXME

print(len(validation_sequences))
print(validation_padded.shape)

# Expected output
# 445
# (445, 120)

445
(445, 120)


In [68]:
label_tokenizer = Tokenizer() # FIXME
label_tokenizer.fit_on_texts(train_labels)

print(label_tokenizer.word_index)

{'sport': 1, 'business': 2, 'politics': 3, 'entertainment': 4, 'tech': 5}


In [69]:
training_label_seq =  label_tokenizer.texts_to_sequences(train_labels) # FIXME
training_label_seq = pad_sequences(training_label_seq)

validation_label_seq =  label_tokenizer.texts_to_sequences(validation_labels) # FIXME
validation_label_seq  = pad_sequences(validation_label_seq)

print(train_labels[0])
print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

print(validation_label_seq[0])
print(validation_label_seq[1])
print(validation_label_seq[2])
print(validation_label_seq.shape)

# Expected output
# [4]
# [2]
# [1]
# (1780, 1)
# [5]
# [4]
# [3]
# (445, 1)

tech
[5]
[2]
[1]
(1780, 1)
[4]
[5]
[3]
(445, 1)


In [72]:
model = tf.keras.Sequential([
# FIXME
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

# Expected Output
# Layer (type)                 Output Shape              Param #   
# =================================================================
# embedding (Embedding)        (None, 120, 16)           16000     
# _________________________________________________________________
# global_average_pooling1d (Gl (None, 16)                0         
# _________________________________________________________________
# dense (Dense)                (None, 24)                408       
# _________________________________________________________________
# dense_1 (Dense)              (None, 6)                 150       
# =================================================================
# Total params: 16,558
# Trainable params: 16,558
# Non-trainable params: 0

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 16)           16000     
_________________________________________________________________
global_average_pooling1d_4 ( (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 150       
Total params: 16,558
Trainable params: 16,558
Non-trainable params: 0
_________________________________________________________________


In [73]:
num_epochs = 15
history = model.fit(train_padded, training_label_seq, epochs = num_epochs, validation_data=(validation_padded, validation_label_seq))

Epoch 1/15
56/56 [==============================] - 0s 7ms/step - loss: 1.7577 - accuracy: 0.2309 - val_loss: 1.7173 - val_accuracy: 0.2270
Epoch 2/15
56/56 [==============================] - 0s 6ms/step - loss: 1.6800 - accuracy: 0.2303 - val_loss: 1.6504 - val_accuracy: 0.2292
Epoch 3/15
56/56 [==============================] - 0s 5ms/step - loss: 1.5855 - accuracy: 0.3309 - val_loss: 1.5873 - val_accuracy: 0.3124
Epoch 4/15
56/56 [==============================] - 0s 6ms/step - loss: 1.4595 - accuracy: 0.3966 - val_loss: 1.5136 - val_accuracy: 0.3865
Epoch 5/15
56/56 [==============================] - 0s 6ms/step - loss: 1.3028 - accuracy: 0.4242 - val_loss: 1.4308 - val_accuracy: 0.3955
Epoch 6/15
56/56 [==============================] - 0s 5ms/step - loss: 1.1407 - accuracy: 0.5242 - val_loss: 1.3407 - val_accuracy: 0.5281
Epoch 7/15
56/56 [==============================] - 0s 6ms/step - loss: 0.9808 - accuracy: 0.7478 - val_loss: 1.2644 - val_accuracy: 0.5596
Epoch 8/15
56/56 [==